In [2]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [3]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [4]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [5]:

dependent=dataset['classification_yes']


In [6]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [16]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid= {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto'], 'class_weight':['dict','balanced'],'C':[0.001,0.01,0.1,1.0,10.0,100.0,1000.0]}

grid=GridSearchCV(SVC(probability=True),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
280 fits failed out of a total of 560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 199, in fit
    y = self._validate_targets(y)
  File "C:\Users\poorn\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 718, in _validate_targets
    self.class_weight_ = compute_class_weight(self.class_weight, classes=cls,

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
                         'class_weight': ['dict', 'balanced'],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1_weighted')

In [20]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [21]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00123501, 0.00072417, 0.00155778, 0.00107923, 0.00117898,
       0.00119972, 0.00109982, 0.0010036 , 0.01716776, 0.01941123,
       0.03002291, 0.02705364, 0.01514745, 0.01771784, 0.02709265,
       0.02305145, 0.00109229, 0.00111842, 0.00087495, 0.00114455,
       0.00063372, 0.00119891, 0.00078788, 0.00100117, 0.00807738,
       0.01832194, 0.02805042, 0.02628632, 0.00835853, 0.01893744,
       0.02929907, 0.02480216, 0.0010035 , 0.00100136, 0.00099969,
       0.00099816, 0.00100203, 0.00080056, 0.00080023, 0.00099969,
       0.00640059, 0.01619949, 0.01920013, 0.01560116, 0.00555215,
       0.01618543, 0.01819921, 0.01500316, 0.00099721, 0.00100226,
       0.00099883, 0.00100307, 0.00099955, 0.00079746, 0.00100131,
       0.00079637, 0.00540023, 0.0134007 , 0.00799799, 0.00860009,
       0.00520153, 0.01320148, 0.00879455, 0.00778708, 0.00100265,
       0.00100298, 0.00039992, 0.00100565, 0.00100164, 0.00099778,
       0.00100384, 0.00100088, 0.00460415, 0

In [22]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[51,  0],
       [ 3, 79]], dtype=int64)

In [23]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        51
           1       1.00      0.96      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [24]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 1.0, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'linear'}: 0.9775556904684072


In [25]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


1.0

In [26]:
table=pd.DataFrame.from_dict(re)

In [27]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001235,0.000393,0.000000,0.000000,0.001,dict,scale,linear,"{'C': 0.001, 'class_weight': 'dict', 'gamma': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,73
1,0.000724,0.000612,0.000000,0.000000,0.001,dict,scale,poly,"{'C': 0.001, 'class_weight': 'dict', 'gamma': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,90
2,0.001558,0.000385,0.000000,0.000000,0.001,dict,scale,rbf,"{'C': 0.001, 'class_weight': 'dict', 'gamma': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,89
3,0.001079,0.000132,0.000000,0.000000,0.001,dict,scale,sigmoid,"{'C': 0.001, 'class_weight': 'dict', 'gamma': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,88
4,0.001179,0.000291,0.000000,0.000000,0.001,dict,auto,linear,"{'C': 0.001, 'class_weight': 'dict', 'gamma': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.005397,0.000801,0.002002,0.000003,1000,balanced,scale,sigmoid,"{'C': 1000.0, 'class_weight': 'balanced', 'gam...",0.981569,0.943699,0.962573,0.981031,0.981217,0.970018,0.015021,31
108,0.004387,0.000477,0.002003,0.000005,1000,balanced,auto,linear,"{'C': 1000.0, 'class_weight': 'balanced', 'gam...",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,25
109,0.011196,0.000401,0.002002,0.000006,1000,balanced,auto,poly,"{'C': 1000.0, 'class_weight': 'balanced', 'gam...",0.963284,0.981014,0.981217,1.000000,0.981217,0.981346,0.011613,15
110,0.005602,0.000492,0.002202,0.000400,1000,balanced,auto,rbf,"{'C': 1000.0, 'class_weight': 'balanced', 'gam...",0.963284,0.981014,1.000000,0.981031,1.000000,0.985066,0.013807,10
